In [40]:
import pandas as pd
import networkx as nx
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import joblib
import spacy
from sympy import symbols, Implies
import faiss
import torch
from langchain_ollama import OllamaLLM
from tqdm import tqdm
import time
import evaluate
import psutil

In [3]:
# Load the chunked legal cases dataset
chunked_cases_df = pd.read_csv("chunked_law_cases.csv")

In [4]:
# Initialize graph
G = nx.DiGraph()

In [5]:
# Load a sentence embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [6]:
# Create node embeddings
chunk_embeddings = model.encode(chunked_cases_df["text"].tolist(), convert_to_tensor=True)

In [7]:
# Add nodes to the graph
for index, row in chunked_cases_df.iterrows():
    G.add_node(row["chunk_id"], text=row["text"], embedding=chunk_embeddings[index])

In [8]:
# Establish logical dependencies between chunks
for i, row in chunked_cases_df.iterrows():
    current_embedding = chunk_embeddings[i].cpu().numpy()
    similarities = cosine_similarity([current_embedding], chunk_embeddings.cpu().numpy())[0]
    
    # Find the top related chunks (excluding itself)
    top_related = np.argsort(similarities)[-6:-1]  # Get top 5 similar chunks
    
    for related_index in top_related:
        related_chunk_id = chunked_cases_df.iloc[related_index]["chunk_id"]
        G.add_edge(row["chunk_id"], related_chunk_id, weight=similarities[related_index])

In [9]:
# # Save graph using pickle (more suitable for complex objects)
# with open("../data/processed/logical_reasoning/graph_data.pkl", "wb") as f:
#     pickle.dump(G, f)

### Logical Reasoning Implementation

In [14]:
def is_logically_relevant(query, chunk_text):
    """
    Rule-based filtering to check if a chunk is logically relevant.
    - Prioritizes legal precedent cases.
    - Ensures the chunk discusses related legal principles.
    """
    keywords = ["precedent", "ruling", "judgment", "legal principle", "interpretation"]
    if any(keyword in chunk_text.lower() for keyword in keywords):
        return True  # Accept if it contains key legal reasoning terms
    return False  # Reject otherwise

In [27]:
# Convert embeddings to NumPy array
embedding_dim = chunk_embeddings.shape[1]  # Get the embedding size
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 (Euclidean) distance index

In [28]:
# Convert PyTorch tensor to NumPy and add to FAISS
faiss_index.add(chunk_embeddings.cpu().numpy())  # Ensure it's a NumPy array

print("✅ FAISS index initialized with", faiss_index.ntotal, "legal case chunks.")

✅ FAISS index initialized with 34982 legal case chunks.


In [29]:
def retrieve_similar(query, top_k=5):
    """
    Retrieves top_k most similar legal case chunks using FAISS.
    """
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()
    
    # FAISS similarity search
    _, indices = faiss_index.search(query_embedding, top_k)

    # Retrieve the actual text of similar chunks
    similar_chunks = chunked_cases_df.iloc[indices[0]]['text'].tolist()
    
    return similar_chunks

In [30]:
def logical_expand_graph_rag(query):
    """
    Expands the context for retrieval by using graph-based reasoning.
    - Retrieves top similar chunks.
    - Expands context using graph neighbors.
    - Applies rule-based filtering for logical relevance.
    """
    similar_chunks = retrieve_similar(query)  # Step 1: Retrieve initial similar chunks
    expanded_context = set(similar_chunks)  # Step 2: Store similar chunks

    # Step 3: Expand using graph-based logical connections
    for chunk in similar_chunks:
        chunk_id = chunked_cases_df[chunked_cases_df['text'] == chunk]['chunk_id'].values[0]
        neighbors = list(G.neighbors(chunk_id))  # Get connected chunks (logical relations)

        for neighbor in neighbors:
            neighbor_text = G.nodes[neighbor]['text']

            # Step 4: Apply symbolic reasoning (filter relevant legal cases)
            if is_logically_relevant(query, neighbor_text):  
                expanded_context.add(neighbor_text)

    return "\n".join(expanded_context)

In [31]:
llm = OllamaLLM(model="llama3.1")

In [36]:
def generate_logically_reasoned_response(query):
    """
    Generates a response using LLaMA with logically expanded retrieval.
    - Retrieves expanded legal context.
    - Uses LLaMA to generate responses based on enriched knowledge.
    """
    expanded_context = logical_expand_graph_rag(query)  # Step 1: Retrieve expanded legal context

    # Step 2: Format the prompt for LLaMA
    prompt = f"""
    Given the following legal case references, provide a logically sound answer to the query.

    Context:
    {expanded_context}

    Query: {query}

    Response:
    """

    # Step 3: Generate response using LLaMA
    response = llm.generate([prompt], max_tokens=500)
    return response

In [37]:
# Load evaluation dataset
questions_df = pd.read_csv("../data/processed/Questions & Answers.csv")

In [39]:
# Select test queries
test_questions = questions_df["question"].sample(3).tolist()

# Iterate with tqdm progress bar
for i, test_query in enumerate(tqdm(test_questions, desc="Generating Responses", unit="query")):
    start_time = time.time()  # Track time per query

    print(f"\n🔹 **Test Query {i+1}:** {test_query}")

    # Generate response
    response = generate_logically_reasoned_response(test_query)

    elapsed_time = time.time() - start_time  # Calculate elapsed time

    print(f"✅ **LLaMA Response:**\n{response}")
    print(f"⏳ Time Taken: {elapsed_time:.2f}s\n{'-'*80}\n")

Generating Responses:   0%|          | 0/3 [00:00<?, ?query/s]


🔹 **Test Query 1:** What is the legal significance of the immunity granted to Attorneys-at-Law regarding statements made in pleadings?


Generating Responses:  33%|███▎      | 1/3 [38:35<1:17:10, 2315.47s/query]

✅ **LLaMA Response:**
generations=[[GenerationChunk(text='I don\'t see a query or response related to the provided text. However, I can answer your question based on the context.\n\nThe legal significance of the immunity granted to Attorneys-at-Law regarding statements made in pleadings is that they are protected from being sued for libel or defamation when making statements in pleadings as part of their professional duty. This means that clients cannot sue their attorneys for mistakes, inaccuracies, or untruths contained in pleadings filed in court.\n\nTo provide a more accurate answer to the query you mentioned at the end ("Query: What is the legal significance of the immunity granted to Attorneys-at-Law regarding statements made in pleadings?"), I would need the actual text of the query. However, based on the provided text, it appears to be related to Sri Lankan case law and constitutional interpretation, which does not seem directly related to the query about attorneys\' immunity.\

Generating Responses:  67%|██████▋   | 2/3 [1:18:14<39:12, 2352.96s/query]

✅ **LLaMA Response:**
generations=[[GenerationChunk(text='Based on the provided text, it appears that in Sri Lanka, the "value" of shares for stamp duty purposes is determined by the Chief Valuer\'s valuation. The text mentions:\n\n"...and is ranked first in the financial offers...per share price of Rs. 13/76 is above the chief valuer’s valuation..."\n\nThis suggests that the Chief Valuer\'s valuation sets a baseline or reference point for determining the value of shares for stamp duty purposes.\n\nAdditionally, the text also refers to another method of determining the value of shares, which involves soliciting offers from interested parties. It mentions:\n\n"The Profile prepared for the Government (P ) had clearly referred to both the properties in question...Moreover, in paragraph 10 of the Agreement between the Government of Sri Lanka and the 2nd respondent, both lands were clearly referred to as the immovable properties in question."\n\nThis implies that when shares are being sold 

Generating Responses: 100%|██████████| 3/3 [1:58:16<00:00, 2365.63s/query]

✅ **LLaMA Response:**
generations=[[GenerationChunk(text="It seems like you've provided a lengthy document with multiple topics discussed. I'll try to answer the question about resolving conflicts between two statutes in tax disputes, as it appears to be the last part of the text.\n\nTo resolve conflicts between two statutes in tax disputes, courts typically follow a hierarchical approach:\n\n1. **Specificity**: If one statute specifically addresses an issue, that statute will prevail over a more general or vague provision.\n2. **Prioritization**: Courts may consider which statute was enacted later and therefore might be intended to supersede the earlier law.\n3. **Purpose**: The court's interpretation should align with the purpose of each statute. If one statute contradicts another but serves a different legislative intent, the court will choose the provision that best achieves its intended goal.\n\nIn tax disputes specifically, courts often consult:\n\n1. **Legislative history**: Ana

### Evaluate Responses Using ROUGE & BERTScore

In [41]:
# Load evaluation metrics
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

In [42]:
# Store evaluation results
evaluation_results = []

In [ ]:
# Process each question with a progress bar
for _, row in tqdm(questions_df.iterrows(), total=len(questions_df), desc="Evaluating Responses", unit="question"):
    question, ground_truth = row["question"], row["answer"]

    # Ensure the inputs are valid strings
    ground_truth = str(ground_truth) if ground_truth else ""
    
    # Measure system performance
    start_time = time.time()
    cpu_before = psutil.cpu_percent(interval=None)
    memory_before = psutil.virtual_memory().percent

    # Generate LLaMA response with logical reasoning
    generated_response = generate_logically_reasoned_response(question)
    
    # Ensure the generated response is valid
    generated_response = str(generated_response) if generated_response else ""

    # Measure system performance after generation
    cpu_after = psutil.cpu_percent(interval=None)
    memory_after = psutil.virtual_memory().percent
    response_time = time.time() - start_time

    # Compute ROUGE scores if response is non-empty
    if generated_response.strip() and ground_truth.strip():
        rouge_scores = rouge.compute(predictions=[generated_response], references=[ground_truth])
        bert_scores = bertscore.compute(predictions=[generated_response], references=[ground_truth], lang="en")
    else:
        rouge_scores = {"rouge1": 0.0, "rouge2": 0.0, "rougeL": 0.0}
        bert_scores = {"precision": [0.0], "recall": [0.0], "f1": [0.0]}

    # Store results
    evaluation_results.append({
        "question": question,
        "response": generated_response,
        "response_time": response_time,
        "cpu_usage": (cpu_before + cpu_after) / 2,
        "memory_usage": (memory_before + memory_after) / 2,
        "rouge_scores": rouge_scores,
        "bert_precision": bert_scores["precision"][0],
        "bert_recall": bert_scores["recall"][0],
        "bert_f1": bert_scores["f1"][0],
    })

    # Print log after each question
    print(f"✅ Processed: {question} | Time: {response_time:.2f}s | "
          f"CPU: {(cpu_before + cpu_after) / 2:.2f}% | "
          f"Memory: {(memory_before + memory_after) / 2:.2f}%")

Evaluating Responses:   0%|          | 0/4 [34:42<?, ?question/s]


ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: generations=[[GenerationChunk(text='The provided text does not discuss how the Supreme Court differentiates between an interlocutory order and a final order in civil appeals.\n\nHowever, based on general knowledge of Sri Lankan law, the Supreme Court has considered this matter in several judgments. Here is a brief overview:\n\nIn cases where the Supreme Court is dealing with appeals from lower courts, it typically distinguishes between two types of orders: interim or interlocutory orders and final orders.\n\nInterim or interlocutory orders are those that decide some aspect of a case but do not f